In [1]:
!pip install google-cloud-aiplatform --upgrade
!pip install kfp==2.4.0 google-cloud-pipeline-components
!pip install gcsfs
!pip install -U google-cloud-aiplatform "shapely>2"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 41.7 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.67.1
    Uninstalling google-cloud-aiplatform-1.67.1:
      Successfully uninstalled google-cloud-aiplatform-1.67.1
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of google-cloud-pipeline-components to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of google-cloud-pipeline-components to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 28.6 MB/s eta 0:00:00
  Created wheel for kfp: filename=kfp-2.4.0-py3-none-any.whl size=542976 sha256=4aadd53ac25457f4e7054073d59c80e93c8d44c636c1c5c0ccd3c339afb1efa7
  Stored in directory: /home/jupyter/.cache/pip/wheels/8f/68/87/2a7654c22807614c76ea1aefcdd62

In [2]:
# ライブラリをインストールした直後はライブラリの更新がカーネルに反映されないことがあるから、カーネルを再起動する
# JupyterNotebookはPythonだけでなくRも実行できる。だから、カーネル（プログラムを実行する頭脳）をPython3に選択する必要がある
# テスト中の無駄な再起動を省くためのif文

import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [1]:
# kfp, google_cloud_pipeline_componentsをインポート
# 同時に、バージョン確認

!python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"
!python3 -c "import google_cloud_pipeline_components; print('google_cloud_pipeline_components version: {}'.format(google_cloud_pipeline_components.__version__))"


KFP SDK version: 2.4.0
google_cloud_pipeline_components version: 2.8.0


In [2]:
# プロジェクトIDの取得

import os
PROJECT_ID = ""
if not os.getenv("IS_TESTING"):
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)


Project ID:  qwiklabs-gcp-00-44faff6c6b58


In [3]:
BUCKET_NAME="gs://" + PROJECT_ID + "-bucket"


In [4]:
# 指定したURLからデータを取得して作業ディレクトリにダウンロード

!wget https://storage.googleapis.com/partner-usecase-bucket/ucase009/Bike-Sharing-Dataset.zip


--2024-11-08 06:10:48--  https://storage.googleapis.com/partner-usecase-bucket/ucase009/Bike-Sharing-Dataset.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.202.207, 74.125.69.207, 64.233.181.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.202.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 279992 (273K) [application/zip]
Saving to: ‘Bike-Sharing-Dataset.zip’

Bike-Sharing-Datase 100%[===================>] 273.43K  --.-KB/s    in 0.004s  

2024-11-08 06:10:48 (69.7 MB/s) - ‘Bike-Sharing-Dataset.zip’ saved [279992/279992]



In [5]:
# dataフォルダ内に解凍
!unzip -q Bike-Sharing-Dataset.zip -d data


In [6]:
# 指定された GCS バケットにコピー

! gsutil cp ./data/hour.csv $BUCKET_NAME


Copying file://./data/hour.csv [Content-Type=text/csv]...
/ [1 files][  1.1 MiB/  1.1 MiB]                                                
Operation completed over 1 objects/1.1 MiB.                                      


In [7]:
! gsutil ls -al $BUCKET_NAME

   1156736  2024-11-08T06:13:16Z  gs://qwiklabs-gcp-00-44faff6c6b58-bucket/hour.csv#1731046396811356  metageneration=1
TOTAL: 1 objects, 1156736 bytes (1.1 MiB)


In [8]:
import os
import pprint as pp
import sys

import pickle
import os
import argparse

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error,mean_absolute_percentage_error
import numpy as np
import pandas as pd

from google.cloud import storage
from google.cloud import aiplatform 


In [ ]:
from typing import NamedTuple

import kfp
from kfp import dsl
from kfp.dsl import (Artifact, Dataset, Input, InputPath, Model, Output,
                        OutputPath, component , ClassificationMetrics , Metrics)

from kfp import compiler
from kfp.components import load_component_from_file

import json
import yaml



In [ ]:
!mkdir components


In [ ]:
@component(
    packages_to_install=["pandas", "pyarrow" , "fsspec" , "gcsfs"],
    base_image="python:3.9",
    output_component_file="./components/download_data.yaml"
)
def download_data(input_data_path : str
                  , input_data_filename : str
                  , downloaded_data : Output[Dataset]):
    
    import pandas as pd
    import os
    
    print(f"input_data_path : {input_data_path}")
    print(f"input_data_filename : {input_data_filename}")
    print(f"downloaded_data : {downloaded_data}")
    print(f"downloaded_data.path : {downloaded_data.path}")
                
    url = os.path.join(input_data_path , input_data_filename)
    
    #read data from GCS location
    data = pd.read_csv(url)
    
    #write to output dataset path
    output_data_uri = downloaded_data.path + ".csv" 
    data.to_csv(output_data_uri 
                , index=False
                , encoding='utf-8-sig')

In [ ]:
@component(
    packages_to_install=["pandas", "pyarrow", "fsspec" , "gcsfs"],
    base_image="python:3.9",
    output_component_file="./components/preprocess_data.yaml"
)
def preprocess_data(train_size : float 
                    , test_size : float
                    , valid_size : float
                    , train_data : Output[Dataset]
                    , valid_data : Output[Dataset]
                    , test_data : Output[Dataset]
                    , input_data:  Input[Dataset]
                   ):
    
    import numpy as np
    import pandas as pd
    
    print(f"train_size : {train_size}")
    print(f"test_size : {test_size}")
    print(f"valid_size : {valid_size}")
    print(f"train_data : {train_data}")
    print(f"valid_data : {valid_data}")
    print(f"test_data : {test_data}")
    print(f"input_data : {input_data}")
    
    data = pd.read_csv(input_data.path + ".csv")
    
    modelling_columns = ["season" 
                     , "yr" 
                     ,"mnth" 
                     ,"hr" 
                     ,"holiday" 
                     , "weekday" 
                     , "workingday" 
                     , "weathersit" 
                     , "temp" 
                     , "atemp" 
                     , "hum" 
                     , "windspeed" 
                     , "casual" 
                     , "registered" 
                     , "cnt"
                    ]
    
    data = data[modelling_columns]

    train_ds, valid_ds, test_ds = np.split(data.sample(frac=1, random_state=42), [int((train_size)*len(data)), int((1-test_size)*len(data))])
    train_ds.to_csv(train_data.path + ".csv" , index=False, encoding='utf-8-sig')
    valid_ds.to_csv(valid_data.path + ".csv" , index=False, encoding='utf-8-sig')
    test_ds.to_csv(test_data.path + ".csv" , index=False, encoding='utf-8-sig')


In [ ]:
@component(
    packages_to_install=["kfp==2.4.0", "pandas", "pyarrow",  "scikit-learn==1.3.2" , "fsspec" , "gcsfs", "click==8.1.7", "docstring-parser==0.16", "kfp-pipeline-spec==0.2.2", "kfp-server-api==2.0.5", "kubernetes==26.1.0", "PyYAML==6.0.2", "requests-toolbelt==0.10.1", "tabulate==0.9.0", "protobuf==3.20.3", "urllib3==1.26.20"]
    , base_image="python:3.9"
    , output_component_file="./components/train.yaml"
)
def train_model(
    train_data:  Input[Dataset],
    model: Output[Model], 
):
    
    print(f"train_data : {train_data}")
    print(f"model : {model}")
    
    from sklearn.ensemble import RandomForestRegressor
    import pandas as pd
    import pickle
    import sklearn

    train_ds = pd.read_csv(train_data.path+".csv")
    my_model = RandomForestRegressor()
    
    target = "cnt"
    
    x_train = train_ds.drop(columns=target, axis=1)
    y_train = train_ds[target]
    
    my_model.fit(x_train , y_train)
    model.metadata["model_name"] = "RandomForestRegressor"
    model.metadata["framework"] = "sklearn"
    model.metadata["framework_version"] = sklearn.__version__
    file_name = model.path + f".pkl"
    
    with open(file_name, 'wb') as file:  
        pickle.dump(my_model, file)


In [ ]:
@component(
    packages_to_install=["pandas", "pyarrow",  "scikit-learn==1.3.2" , "fsspec" , "gcsfs"]
    , base_image="python:3.9"
    , output_component_file="./components/evaluate_model.yaml"
)
def evaluate_model(
    test_data:  Input[Dataset],
    model: Input[Model], 
    target_column_name : str ,
    deployment_metric : str ,
    deployment_metric_threshold : float ,
    kpi: Output[Metrics]
)-> NamedTuple(
    "Outputs",
    [
        ("deploy_flag", str),  # Return parameter.
    ],
):
    
    print(f"test_data : {test_data}")
    print(f"model : {model}")
    print(f"kpi : {kpi}")
    print(f"deployment_metric : {deployment_metric}")
    print(f"deployment_metric_threshold : {deployment_metric_threshold}")
    
    from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error,mean_absolute_percentage_error
    import pandas as pd
    import pickle
    import numpy as np
    import json
    
    test_ds = pd.read_csv(test_data.path+".csv")
    target = target_column_name
    
    x_test = test_ds.drop(columns=target, axis=1)
    y_test = test_ds[target]
    
    print(f"model.path : {model.path}")
    file_name = model.path + f".pkl"
    print(f"file_name : {file_name}")
    #model = pickle.loads(file_name)
    with open(file_name, 'rb') as file:  
        model = pickle.load(file)
    
    y_pred = model.predict(x_test)
    r2 = r2_score(y_true=y_test, y_pred=y_pred)
    mae = mean_absolute_error(y_true=y_test, y_pred=y_pred)
    mse = mean_squared_error(y_true=y_test, y_pred=y_pred)
    mape = mean_absolute_percentage_error(y_true=y_test, y_pred=y_pred)
    rmse = np.sqrt(mse)
    
    model_metrics = {"r2" : r2 
                     , "mae" : mae 
                     , "mape" : mape 
                     , "mse" : mse 
                     , "rmse" : rmse
                    }
    
    print(f"Adjusted_R2 : {r2}")
    print(f"Mean Absolute Error : {mae}")
    print(f"Mean Absolute Percentage Error : {round(mape,4)*100}%")
    print(f"Mean Squared Error : {mse}")
    print(f"Root Mean Squared Error : {rmse}")
    
    kpi.log_metric("Adjusted_R2", float(r2))
    kpi.log_metric("Mean Absolute Error", float(mae))
    kpi.log_metric("Mean Absolute Percentage Error", float(mape))
    kpi.log_metric("Mean Squared Error", float(mse))
    kpi.log_metric("Root Mean Squared Error", float(rmse))
    
    actual_metric_value = model_metrics.get(deployment_metric)
    
    if actual_metric_value >= deployment_metric_threshold:
        deploy_flag = "True"
    else:
        deploy_flag = "False"
        
    return (deploy_flag,)


In [ ]:
@component(
    packages_to_install=["pandas", "pyarrow",  "scikit-learn==1.3.2" , "fsspec" , "gcsfs" , "google-cloud-aiplatform"]
    , base_image="python:3.9"
    , output_component_file="./components/register_model.yaml"
)
def register_model(
    serving_container_uri : str ,
    project_id : str ,
    region: str,
    model_name : str , 
    model: Input[Model], 
)-> NamedTuple(
    "Outputs",
    [
        ("model_resource_name", str),  # Return parameter.
    ],
):
    
    print(f"serving_container_uri : {serving_container_uri}")
    print(f"project_id : {project_id}")
    print(f"region : {region}")
    print(f"model : {model}")
    
    from google.cloud import aiplatform
    
    print(f"model.uri : {model.uri[:-5]}")
    
    aiplatform.init(project = project_id , location=region)
    model = aiplatform.Model.upload(display_name= model_name ,
                                    artifact_uri=model.uri[:-5],
                                    serving_container_image_uri=serving_container_uri)
    return (model.resource_name,)

In [ ]:
@component(
    packages_to_install=["kfp==2.4.0", "pandas", "pyarrow",  "scikit-learn==1.3.2" , "fsspec" , "gcsfs", "google-cloud-aiplatform", "click==8.1.7", "kfp-pipeline-spec==0.2.2", "kfp-server-api==2.0.5", "kubernetes==26.1.0", "PyYAML==6.0.2", "requests-toolbelt==0.10.1", "tabulate==0.9.0", "protobuf==3.20.3", "urllib3==1.26.20", "numpy==1.24.4", "google-cloud-pipeline-components==2.8.0"]
    , base_image="python:3.9"
    , output_component_file="./components/deploy_model.yaml"
)
def deploy_model(
    model_resource_name : str ,
    project_id : str ,
    region: str
)-> NamedTuple(
    "Outputs",
    [
        ("endpoint_resource_name", str),  # Return parameter.
    ],
):
    
    print(f"model_resource_name : {model_resource_name}")
    print(f"project_id : {project_id}")
    print(f"region : {region}")
    
    from google.cloud import aiplatform
    
    aiplatform.init(project = project_id , location=region)
    
    model = aiplatform.Model(model_resource_name)
    endpoint = model.deploy(machine_type="n1-standard-4",
                        min_replica_count=1,
                        max_replica_count=1)
    
    return (endpoint.resource_name,)


In [ ]:
shell_output = !gcloud auth list 2>/dev/null
SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()
print("Service Account:", SERVICE_ACCOUNT)
print("Project ID: ", PROJECT_ID)
print("staging_bucket_uri: ",BUCKET_NAME)
print("input_data_path: ",BUCKET_NAME)


In [ ]:
%%writefile config.json
{
    "project":"ENTER PROJECT_ID HERE",
    "region":"us-central1",
    "service_account":"ENTER SERVICE_ACCOUNT HERE",
    "staging_bucket_uri":"ENTER BUCKET_NAME HERE",
    "pipeline_name":"tabular-data-regression-kfp-cicd-pipeline",
    "pipeline_package_path":"tabular-data-regression-kfp-cicd-pipeline.json",
    "input_data_path":"ENTER BUCKET_NAME HERE",
    "input_data_filename":"hour.csv",
    "target_column_name":"cnt",
    "train_size":0.8,
    "test_size":0.1,
    "valid_size":0.1,
    "deployment_metric":"r2",
    "deployment_metric_threshold":0.8,
    "model_name":"model_tabular_regression",
    "serving_container_uri":"us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-3:latest"
}

In [ ]:
%%writefile build_pipeline.py

import json
import yaml

import kfp
from kfp import dsl
from kfp import compiler
from kfp.components import load_component_from_file

download_data = load_component_from_file("./components/download_data.yaml")
preprocess_data = load_component_from_file("./components/preprocess_data.yaml")
train_model = load_component_from_file("./components/train.yaml")
evaluate_model = load_component_from_file("./components/evaluate_model.yaml")
register_model = load_component_from_file("./components/register_model.yaml")
deploy_model = load_component_from_file("./components/deploy_model.yaml")

#read configuration from file
with open("config.json") as json_file:
    config = json.load(json_file)
    
PIPELINE_NAME = config.get("pipeline_name")
PACKAGE_PATH = config.get("pipeline_package_path")
BUCKET_URI = config.get("staging_bucket_uri")
PIPELINE_ROOT = "{}/pipeline_root/kfp_tabular_data_regression".format(BUCKET_URI)
print(f"PIPELINE_ROOT :{PIPELINE_ROOT}")

@dsl.pipeline(
    # Default pipeline root. You can override it when submitting the pipeline.
    pipeline_root=PIPELINE_ROOT,
    # A name for the pipeline. Use to define the pipeline Context.
    name=PIPELINE_NAME,
    
)
def pipeline(project: str = "",
 region: str = "",
 service_account: str = "",
 staging_bucket_uri: str = "",
 pipeline_name: str = "",
 pipeline_package_path: str = "",
 input_data_path: str = "",
 input_data_filename: str = "",
 target_column_name: str = "",
 train_size: float = 0.8,
 test_size: float = 0.1,
 valid_size: float = 0.1,
 hypertune_container_image_uri: str = "" ,
 hypertune_machine_type: str = "",
 hypertune_machine_replica_count: int = 1 ,
 hypertune_max_trial_count: int = 1 ,
 hypertune_parallel_trial_count: int = 1 ,
 hypertune_metric : str = "" ,
 hypertune_metric_objective : str = "" ,
 hypertune_job_name: str = "" ,
 deployment_metric: str = "" ,
 deployment_metric_threshold: float = 0.8 ,
 serving_container_uri : str = "" ,
 model_name : str = "", 
 user_email : str = "",
 monitoring_job_name : str = "" ,
 predict_instance_schema_uri : str = ""
):
    
    download_data_op = download_data(input_data_path = input_data_path 
                                     , input_data_filename = input_data_filename
                                    )
    
    
    preprocess_data_op = preprocess_data(train_size = train_size
                                         , test_size = test_size
                                         , valid_size = valid_size
                                         , input_data = download_data_op.outputs["downloaded_data"])
    
    train_model_op = train_model(train_data = preprocess_data_op.outputs["train_data"])
    
    
    evaluate_model_op = evaluate_model(test_data = preprocess_data_op.outputs["test_data"]
                                       ,model = train_model_op.outputs["model"]
                                       ,target_column_name = target_column_name
                                       ,deployment_metric = deployment_metric 
                                       ,deployment_metric_threshold = deployment_metric_threshold
                                      )
    
    with dsl.If(evaluate_model_op.outputs["deploy_flag"] == "True"):
        
        register_model_op = register_model(serving_container_uri = serving_container_uri 
                                       , model = train_model_op.outputs["model"]
                                       , model_name = model_name
                                       , project_id = project 
                                       , region = region)
        
        #deploy only if metric value exceeds deployment threshold
        deploy_model_op = deploy_model(model_resource_name = register_model_op.outputs["model_resource_name"] 
                                   , project_id = project 
                                   , region = region)
    
compiler.Compiler().compile(
    pipeline_func=pipeline
    , package_path=PACKAGE_PATH
)


In [ ]:
%%writefile run_pipeline.py

from google.cloud import aiplatform
import yaml
import json

with open("config.json") as json_file:
    config = json.load(json_file)
    
SERVICE_ACCOUNT = config.get("service_account")
DISPLAY_NAME = config.get("pipeline_name")
PACKAGE_PATH = config.get("pipeline_package_path")
BUCKET_URI = config.get("staging_bucket_uri")
PIPELINE_ROOT = "{}/pipeline_root/kfp_tabular_data_regression".format(BUCKET_URI)
print(f"PIPELINE_ROOT :{PIPELINE_ROOT}")

job = aiplatform.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path=PACKAGE_PATH,
    pipeline_root=PIPELINE_ROOT,
    parameter_values=config,
)

job.submit(service_account = SERVICE_ACCOUNT)


In [ ]:
!python3 build_pipeline.py

In [ ]:
!python3 run_pipeline.py

In [ ]:
! gsutil cp ./config.json $BUCKET_NAME
! gsutil cp ./tabular-data-regression-kfp-cicd-pipeline.json $BUCKET_NAME
! gsutil acl ch -u AllUsers:R $BUCKET_NAME/config.json
! gsutil acl ch -u AllUsers:R $BUCKET_NAME/tabular-data-regression-kfp-cicd-pipeline.json